In [ ]:
#First we will get the twtter data
# 2nd step is preprocessing since machine cant understand text we have to convert it to numeric form
# 3rd step is Train test Split in this we will train our data and test its accuracy.
# 4th step is to Feed the data to machine learning model.
# Atlast whenever we feed a new tweet to our model it will tell wether the tweet is positive or negative.

# Data is used from kaglle but the data is very large and will take a lot of time to upload so instead using api for kaggle data

In [ ]:
#Install Kaggle Library
! pip install kaggle

In [ ]:
#Uploading Kaglle. json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json #Allowing reading and writing the kaggle file from API

In [ ]:
#Api to fetch dataset from kaggle
#copying the APi command from kaggle and pasting it here
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 80% 65.0M/80.9M [00:00<00:00, 165MB/s]
100% 80.9M/80.9M [00:00<00:00, 174MB/s]


In [ ]:
#extracting compress dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip' #Path of the zip file where the data set is

with ZipFile(dataset, 'r') as zip: #with open the zipfile in read mode and name the file as zip
  zip.extractall() #extract all the data set from the zip file
  print('Data set is extracted') #printing a confirmation message

Data set is extracted


In [ ]:
import numpy as np #numpy library used to work on arrays
import pandas as pd #pandas used to work with dataframes such as tables or spreadsheets
import re #re stands for regular expression and it is used for string manipulation and pattern matching
from nltk.corpus import stopwords #stopwords are the words which are not required by the model so we remove it from dataset.
from nltk.stem.porter import PorterStemmer #used to reduce words to their base such as running and ran can both be reduced to run
from sklearn.feature_extraction.text import TfidfVectorizer #converting text to numerical vectors Tfidf stands for term frequency inverse document frequency
from sklearn.model_selection import train_test_split #Used to train andtest the data
from sklearn.linear_model import LogisticRegression # It is used when model is working with classification such as(positive tweet or negative)
from sklearn.metrics import accuracy_score #It is a matrix used to test accuracy of the model

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#Data Processing
#Loading the data from csv file to pandas dataframes
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO.8859.1')
#/content/training.1600000.processed.noemoticon.csv Path of the csv file
#Encoding helps converting the textual data to binary format

In [ ]:
#checking the number of rows and columns
twitter_data.shape

(1600000, 7)

In [ ]:
#printing first 5 tweets of data frames
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
#renaming the columns
column_names = ['target' , 'id', 'date' , 'flag' , 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names= column_names ,encoding='ISO.8859.1') #Renaming the columns of the dataset

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#checking the distribution of taget columns
#Check how many positive or negative tweets are there

twitter_data['target'].value_counts() #Both the values are equal (Equal Distribution)

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
#Changing the label of positive tweet from 4 to 1
twitter_data.replace({'target':{4:1}}, inplace=True) #inplace = true means that the new replace would be updated in dataset

In [ ]:
#Stemming the dataset reducing the words to root words.
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #Remove everything from the input tweet which are notalphabeother then alphabets or blank spaces
  stemmed_content = stemmed_content.split() #putting the words in the form of a list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #This line checks the text and apply stemming on all the words other then the stopwords.
  stemmed_content = ' ' . join(stemmed_content) #Combining all the words together in a particular tweet

  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming) #stemming the text column of thw twitter Data
#Since we are only concerned with text then apply stemming on text column only.

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
#Seperating the text and target as theses 2 are need to know if tweet is positive or negative
#Target column contains 0 or 1 indicating if any tweet is positive or negative
#0 for negative
#1 for positive

X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
#Splitting the data to training data an test data
#These 4 variables will store the text and target for training and testing data.
#X_train will store text use in training
#X_test will store the text use in testing data
#Y_train will store target of the training data
#Y_test will store the target of the testing data

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,stratify=Y,random_state=2)

#Here 0.1 means 10% data will be used for testing the model and 90% would be used for training the model
#Stratify = Y means equal distribution of positive and negative tweets in training
#random_state=2 means we have set the seed of random_state to 2 ensuring we will always get the same split.


In [ ]:
# Converting the textual data to numerical data

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train) #It will convert the training data from text to numeric value[matrix]
X_test=vectorizer.transform(X_test) #It will convert Testing data from text to numeric value We will not use fit_transform in this as we dont want model to learn something extra

In [ ]:
print(X_train) #All training data converted from text to numeric value

  (0, 182979)	0.5043398323876811
  (0, 124178)	0.5009098608163001
  (0, 191949)	0.23658323758073335
  (0, 39517)	0.5442402674140161
  (0, 480730)	0.20791176343014625
  (0, 253306)	0.20989916743831274
  (0, 140414)	0.23510973174492172
  (1, 96496)	0.24756740179431355
  (1, 391527)	0.37884912323076886
  (1, 150159)	0.43166246378558804
  (1, 211616)	0.4650971855507986
  (1, 256084)	0.2324974076628597
  (1, 163657)	0.1790916056140654
  (1, 139329)	0.5535404239764236
  (2, 25322)	0.29831434367036275
  (2, 478718)	0.30928207079919917
  (2, 52422)	0.2744074166366401
  (2, 252766)	0.2691390741081691
  (2, 13488)	0.2746918607123101
  (2, 181827)	0.22667234804878209
  (2, 84950)	0.228293337690162
  (2, 149200)	0.6990451071142899
  (3, 443848)	0.4086851938350052
  (3, 105553)	0.912675414558782
  (4, 258806)	0.20025436326440765
  :	:
  (1439997, 306114)	0.17019026786338842
  (1439997, 406947)	0.15350608814882138
  (1439997, 4141)	0.18002122839570336
  (1439997, 324019)	0.1662086339046958
  (143999

In [ ]:
#Training the ML Model
#Logistic Regression

#It creates a logistic regression model a model which is used for the classfication

model=LogisticRegression(max_iter=10000) # Max times the model will go through data.


In [ ]:
model.fit(X_train,Y_train) #Model Leraning

LogisticRegression(max_iter=10000)

In [ ]:
#Accuracy of training data model
X_train_prediction=model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('Accuracy of training data is : ',training_data_accuracy)

Accuracy of training data is :  0.8098618055555555


In [ ]:
#Saving the model
import pickle #Used to save Ml model


In [ ]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb')) #Storing(Dumping) the model


In [ ]:
#using the model for futrue prediction

#Loading the saved model

loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new=X_test[13]
print(Y_test[13])

prediction=loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0): #Prediction returns the value
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet


In [ ]:
New = input('Enter your tweet here: ')
Stemmed_Tweet = stemming(New)  # Assuming stemming function returns a list of stemmed words
print(Stemmed_Tweet)
Transforming = vectorizer.transform([Stemmed_Tweet])
print(Transforming)
prediction=loaded_model.predict(Transforming)
print(prediction)
if(prediction[0]==0): #Prediction returns the value in for of
  print('Negative Tweet')
else:
  print('Positive Tweet')

Enter your tweet here: I am sad today 
sad today
  (0, 443848)	0.6399309884747669
  (0, 377248)	0.7684323847871768
[0]
Negative Tweet
